## use local files and online marketing news to build a situational awareness model

In [ ]:
# collect data from local folders, cloud storages, and online informations
# local data and cloud data with business performance, analytics, and business target
# online informations used for reduce the hallucination and improve insightful and useful contents for LLM-generated strategies, recommend decisions

In [ ]:
# access for local folders
    ## cloud storages, and online informations

# process the text into tokens

# generate the content with summary, strategies recommendations

# store it in sessions, multiple cells

# Access to loca folders
## pdf only

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from typing import List, Dict, Optional
from pathlib import Path
import tiktoken
import warnings

In [61]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
PDF_FOLDER_PATH = os.getenv("PDF_FOLDER_PATH")
print(PDF_FOLDER_PATH)

/Users/eshan/PycharmProjects/FastAPIProject2/test_data/pdf_files


In [41]:
# extract text from PDF files using pdfplumber

import pdfplumber

class TextExtractor:
    def __init__(self, folder_path):
        self.folder_path = Path(folder_path)

    def extract_text_from_pdf(self, pdf_file):
        text = ""
        path = Path(pdf_file)

        if not path.exists() or path.suffix.lower() != ".pdf":
            raise FileFoundError(f"File {path} is not a valid PDF.")

        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text.strip()


    def extract_text_from_pdfs(self):
        pdf_texts = {}
        for pdf_file in self.folder_path.glob("*.pdf"):
            try:
                text = self.extract_text_from_pdf(str(pdf_file))
                pdf_texts[pdf_file.name] = text
            except Exception as e:
                print(f"Failed to process {pdf_file}: {e}")

        return pdf_texts




In [42]:
extractor = TextExtractor(PDF_FOLDER_PATH)
texts = extractor.extract_text_from_pdfs()
for filename, content in texts.items():
    print(f"{filename}: \n{content[:1000]}...\n")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Trends_Artificial_Intelligence.pdf: 
Trends –
Artificial Intelligence
BOND
May 2025
Trends – Artificial Intelligence (AI)
May 30, 2025
Mary Meeker / Jay Simons / Daegwon Chae / Alexander Krey
Context
We set out to compile foundational trends related to AI. A starting collection of several disparate datapoints turned into this beast.
As soon as we updated one chart, we often had to update another –a data game of whack-a-mole…
a pattern that shows no sign of stopping…and will grow more complex as competition
among tech incumbents, emerging attackers and sovereigns accelerates.
Vint Cerf, one of the ‘Founders of the Internet,’ said in 1999, ‘…they say a year in the Internet business is like a dog year –
equivalent to seven years in a regular person's life.’ At the time, the pace of change catalyzed by the internet was unprecedented.
Consider now that AI user and usage trending is ramping materially faster…and the machines can outpace us.
The pace and scope of change related to the artific

# Tokenize the extracted texts

In [45]:
from openai import OpenAI
import tiktoken

def chunk_tokens(text, model="gpt-4", max_tokens = 500):
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)

    chunks = []

    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i: i+ max_tokens]
        chunks.append(encoding.decode(chunk))
        return chunks


In [46]:

# import tiktoken
tokens = chunk_tokens(content)
print(f"Token count: {len(tokens)}")
print(f"Tokens: {tokens[:20]}")

Token count: 1
Tokens: ['2025/6/6 下午2:33 The Forrester Wave™: Data Management For Analytics Platforms, Q2 2025 | 001a000001AEj6nAAD | 53381efb\nWAVE REPORT\nThe Forrester Wave™: Data\nManagement For Analytics\nPlatforms, Q2 2025\nThe 11 Providers That Matter Most And How They Stack Up\nApr 14, 2025 • 17 min read\nNoel Yuhanna\nVP, Principal Analyst\nWith contributors:\nAaron Katz, Emily Doherty and Jen Barton\nShare\nIn our evaluation of data management for analytics (DMA) platform\nproviders, we identified the most significant ones and researched,\nanalyzed, and scored them. This report shows how each provider\nmeasures up and helps you select the right one for your needs.\nhttps://reprint.forrester.com/reports/the-forrester-wavetm-data-management-for-analytics-platforms-q2-2025-53381efb/index.html 1/22\n2025/6/6 下午2:33 The Forrester Wave™: Data Management For Analytics Platforms, Q2 2025 | 001a000001AEj6nAAD | 53381efb\nAI-Driven Automation And Intelligence Are Transforming\nThe DMA 

# Get Embeddings

In [47]:
def get_embeddings(text_chunks, model="text-embedding-ada-002"):
    embeddings=[]
    for chunk in text_chunks:
        response = openai.embeddings.create(input=chunk, model=model)
        embeddings.append(response.data[0].embedding)
    return embeddings

# Connect OpenAI-API

In [56]:
import openai
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# Setting Prompts for
## - Summary
## - Insights and trends

In [57]:
STRATEGY_PROMPT = """
You are a business strategy expert.
Based on the following:
1. Summary of internal documents: {local_summary}
2. Online insights and trends: {online_summary}

Generate a strategic recommendation, including:
- Risk and Opportunities
- Suggested Actions
- Market Positioning Ideas
"""

# Content generation

In [ ]:
class Generator:
    def __init__(self, local_embeddings=None, outsource_embeddings=None, cloud_embeddings=None, trend_prediction=None, model="gpt-4"):
        self.local_embeddings = local_embeddings
        self.outsource_embeddings = outsource_embeddings
        self.cloud_embeddings = cloud_embeddings
        self.trend_prediction = trend_prediction
        self.status_local_embeddings = bool(local_embeddings)
        self.status_outsource_embeddings = bool(outsource_embeddings)
        self.status_cloud_embeddings = bool(cloud_embeddings)
        self.status_trend_prediction = bool(trend_prediction)
        self.model = "gpt-4"

    def generate_strategies(self):
        if not any([self.status_local_embeddings, self.status_outsource_embeddings, self.status_cloud_embeddings, self.status_trend_prediction]):
            raise ValueError("No valid data provided to generate strategy")

        prompt = STRATEGY_PROMPT.format(
            local_summary=self.local_embeddings or "N/A",
            online_summary=self.outsource_embeddings or "N/A",
            cloud_embeddings=self.cloud_embeddings or "N/A",
            trend_prediction=self.trend_prediction or "N/A"
        )
        try:
            response = openai.chat.completions.create(
                model = self.model,
                messages = [{"role":"user", "content": prompt}],
                temperature=0.4
            )
            return response.choices[0].message.content
        except Exception as e:
            return F"Error during strategy generation: {e}"

In [ ]:
market_data = [{
    "predicting_close": 150.0
}]

In [65]:
OUTSOURCE_FOLDER_PATH = os.getenv("OUTSOURCE_FOLDER_PATH")
CLOUD_FOLDER_PATH = os.getenv("CLOUD_FOLDER_PATH")
print(OUTSOURCE_FOLDER_PATH)
print(CLOUD_FOLDER_PATH)


/Users/eshan/PycharmProjects/FastAPIProject2/test_data/outsource_files
/Users/eshan/PycharmProjects/FastAPIProject2/test_data/cloud_files


In [66]:

local_loader = TextExtractor(PDF_FOLDER_PATH)
outsource_loader = TextExtractor(OUTSOURCE_FOLDER_PATH)
cloud_loader = TextExtractor(CLOUD_FOLDER_PATH)

In [ ]:
generator = generator = Generator(
    local_embeddings="Sales dropped 5% QoQ in internal data.",
    outsource_embeddings="Market shows recovery signs in APAC region.",
    cloud_embeddings="Cloud costs increased 20% in Q2.",
    trend_prediction="Predicted upward trend for B2B subscriptions next quarter."
)


result = generator.generate_strategies()
print(result)

In [67]:
generator = Generator(
    local_embeddings=local_loader.extract_text_from_pdfs(),
    outsource_embeddings=outsource_loader.extract_text_from_pdfs(),
    cloud_embeddings=cloud_loader.extract_text_from_pdfs(),
    trend_prediction=[{"predicting_close": 150.0}]
)

result = generator.generate_strategies()
print(result)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Error during strategy generation: Error code: 429 - {'error': {'message': 'Request too large for gpt-4 in organization org-nEVCynm6tT0ITy8gMQfcjVRP on tokens per min (TPM): Limit 10000, Requested 82524. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


In [ ]:
# Import libraries
# data_loader.py
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import PyPDF2
import fitz  # PyMuPDF
from pathlib import Path
from typing import List, Dict, Optional
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# DataLoader class to read files from a directory
class TextDataset(Dataset):
    def __init__(self, pdf_directory: str, tokenizer_name: str = "bert-base-uncased", max_length: int = 512, chunk_size: int = 1000, overlap: int= 100, pdf_reader: str = "pymupdf"):
        self.pdf_directory = Path(pdf_directory)
        self.tokenizer_name = tokenizer_name
        self.max_length = max_length
        self.chunk_size = chunk_size
        self.opverlap = overlap
        self.pdf_reader = pdf_reader

        # Initialize tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Find all PDF files
        self.pdf_files = list(self.pdf_directory.glob("**/*.pdf"))
        logger.info(f"Found {len(self.pdf_files)} PDF files in {self.pdf_directory}")

        # Extract and chunk all texts from the PDF files
        self.text_chunks = []
        self._process_all_pdfs()

        logger.info(f"Extracted {len(self.text_chunks)} text chunks from PDF files.")

    def _extract_text_pymupdf(self, pdf_path: Path) -> str:
        try:
            doc = fitz.open(pdf_path)
            text = ""
            for page in doc:
                text += page.get_text()
            doc.close()
            return text
        except Exception as e:
            logger.error(f"Error reading {pdf_path} with PyMuPDF: {e}")
            return ""
    def _extract_text_pypdf2(self, pdf_path: Path) -> str:
        try:
            with open(pdf_path, "rb") as file:
                reader = PyPDF2.PdfReader(file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() or ""
            return text
        except Exception as e:
            logger.error(f"Error reading {pdf_path} with PyPDF2: {e}")
            return ""

    def _extract_text_from_pdf(self, pdf_path: Path) -> str:
        if self.pdf_reader == "pymupdf":
            return self._extract_text_pymupdf(pdf_path)
        elif self.pdf_reader == "pypdf2":
            return self._extract_text_pypdf2(pdf_path)
        else:
            raise ValueError(f"Unsupported PDF reader: {self.pdf_reader}.")
    def _chunk_text(self, text: str) -> List[str]:
        # Split text into chunks with overlap
        if len(text) <= self.chunk_size:
            return [text]
        chunks = []
        start = 0

        while start < len(text):
            end = start + self.chunk_size
            chunk = text[start:end]
            if end < len(text):
                # try to break at word boundaries
                last_space = chunk.rfind(" ")
                if last_space > self.chunk_size * 0.8:  # if the last space is within 80% of chunk size, break there
                    chunk = chunk[:last_space]
                    end = start + last_space

                chunks.append(chunk.strip())
                if end >= len(text):
                    break
                start = end - self.opverlap
        return chunks

    def _clean_text(self, text: str) -> str:
        # Remove extra spaces and newlines
        return ' '.join(text.split())
        # Remove common PDF artifacts
        text = text.replace("\x00", " ")  # Null bytes
        text = text.replace("\ufffd", " ") # Replace characters

        return text

    def _process_all_pdfs(self):
        for pdf_file in self.pdf_files:
            logger.info(f"Processing {pdf_file.name}")

            # Extract text
            raw_text = self._extract_text_from_pdf

            if not raw_text.strip():
                logger.warning(f"No text extracted from {pdf_file.name}")
                continue

            # clean text
            clean_text = self._clean_text(raw_text)

            # Create chunks
            chunks = self._chunk_text(clean_text)

            # Store chunks with metadata
            for i, chunk in enumerate(chunks):
                if len(chunk.strip()) > 50:
                    self.text_chunks.append({
                        "text": chunk,
                        "source_file": pdf_file.name,
                        "chunk_id": i,
                        "total_chunks": len(chunks)
                    })
    def __len__(self):
        return len(self.text_chunks)
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        chunk_data = self.text_chunks[idx]
        text = chunk_data["text"]

        # Tokenize text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length = self.max_length,
            return_tensors='pt'
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "text": text,
            "source_file": chunk_data["source_file"],
            "chunk_id": chunk_data["chunk_id"],
            "total_chunks": chunk_data["total_chunks"]
        }

In [ ]:
class PDFTextDataLoader:
    # Wrapper class for creating DataLoader with PDF text data
    def __init__(
        self,
        pdf_directory: str,
        tokenizer_name: str = "bert-base-uncased",
        batch_size: int = 8,
        max_length: int = 512,
        chunk_size: int = 1000,
        overlap: int = 100,
        shuffle: bool = True,
        num_workers: int = 0,
        pdf_reader: str = "pymupdf"
        )

    def get_dataloader(self):
        return self.dataloader

    def get_dataset_info(self):
        return {
            "total_chunks": len(self.dataset),
            "total_pdf_files": len(self.dataset.pdf_files),
            "tokenizer": self.dataset.tokenizer.name_or_path,
            "max_length": self.dataset.max_length,
            "chunk_size": self.dataset.chunk_size
        }

In [28]:
if __name__ == "__main__":
    pdf_dir = "path/to/directory"

    # create the dataloader
    pdf_dataloader = PDFTextDataLoader(
        pdf_directory=pdf_dir,
        tokenizer_name="bert-base-uncased",
        batch_size=8,
        max_length=512,
        chunk_size=1000,
        overlap=100,
        shuffle=True,
        num_workers=0,
        pdf_reader="pymupdf"
    )

    # Get dataset information
    info = pdf_dataloader.get_dataset_info()
    print("Dataset Info:")
    for key, val in info.items():
        print(f" {key}: {value}")

    # Get the actual DataLoader
    dataloader = pdf_dataloader.get_dataloader()

    print("\nTesting DataLoader:")
    for i, batch in enumerate(dataloader):
        print(f"Batch {i+1}:")
        print(" Input IDs:", batch["input_ids"].shape)
        print(" Attention Mask:", batch["attention_mask"].shape)
        print(" Text Sample:", batch["text"][0])
        print(" Source File:", batch["source_file"][0])
        print(" Chunk ID:", batch["chunk_id"][0])
        print(" Total Chunks:", batch["total_chunks"][0])
        if i >= 2:  # Limit to first 3 batches for testing
            break

    print(f"\nTotal batches: {len(dataloader)}")




In [29]:
# text_extractor.py
import textract
import pandas as pd
from pathlib import Path

class TextExtractor:
    def extract(self, file_path: Path):
        try:
            ext = file_path.suffix.lower()
            if ext in ['.txt']:
                return file_path.read_text(encoding='utf-8', errors='ignore')
            elif ext in ['.docx', '.doc', '.pdf']:
                return textract.process(str(file_path)).decode('utf-8')
            elif ext in ['.csv', 'xlsx']:
                df = pd.read_csv(file_path) if ext == '.csv' else pd.read_excel(file_path)
                return df.to_string(index=False)
        except Exception as e:
            print(f"Failed to extract {file_path}: {e}")
            return ""

In [30]:
# embedder.py
from sentence_transformers import SentenceTransformer
import numpy as np

class Embedder:
    def __init__(self, model_name: str="all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)
    def embed_text(self, texts: str):
        return self.model.encode(texts, convert_to_numpy=True)   # convert to tensor or numpy array, which could have better performance?
    def embed_from_files(self, data: list[tuple[str, str]]) -> dict:
        texts = [text for _, text in data]
        embeddings = self.embed_text(texts)
        return {filename: emb for (filename, _), emb in zip(data, embeddings)}

In [31]:
BASE_FOLDER = "/Users/eshan/PycharmProjects/FastAPIProject2/test_data/"

In [37]:

# from embedding.data_loader import DataLoader
# from embedding.embedder import Embedder


# Load data
loader = DataLoader(BASE_FOLDER)
documents = loader.read_all_files("pdf_files", [".pdf"])

# Generate embeddings
embedder = Embedder()
embeddings = embedder.embed_from_files(documents)

# Show results
for filename, vector in embeddings.items():
    print(f"{filename} -> vector shape: {vector.shape}")

print(embeddings.items())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/eshan/PycharmProjects/FastAPIProject2/test_data/pdf_files/Trends_Artificial_Intelligence.pdf -> vector shape: (384,)
/Users/eshan/PycharmProjects/FastAPIProject2/test_data/pdf_files/The Forrester Wave™_ Data Management For Analytics Platforms, Q2 2025 _ 001a000001AEj6nAAD _ 53381efb.pdf -> vector shape: (384,)
dict_items([('/Users/eshan/PycharmProjects/FastAPIProject2/test_data/pdf_files/Trends_Artificial_Intelligence.pdf', array([-8.19380581e-02, -9.56653133e-02,  2.03390829e-02, -7.09827393e-02,
        4.94498648e-02, -2.82026492e-02, -6.52570161e-04,  4.33343947e-02,
       -2.22628247e-02,  3.81168909e-02, -4.00488749e-02,  5.40989228e-02,
        2.98074316e-02, -2.43630446e-02,  1.49037605e-02,  8.60110484e-03,
       -4.12945300e-02, -9.14109126e-02, -3.57101411e-02, -6.09907582e-02,
        2.11805827e-03,  1.57726090e-02,  1.58908404e-03, -5.55848442e-02,
       -1.41519662e-02,  5.78985922e-02,  1.23470370e-02, -5.73277175e-02,
       -2.80143097e-02, -1.57074835e-02,

In [ ]:
print(embeddings.items[0])

In [6]:
def get_embeddings(text_chunks, model="text-embedding-ada-002"):
    embeddings=[]
    for chunk in text_chunks:
        response = openai.embeddings.create(input=chunk, model=model)
        embeddings.append(response.data[0].embedding)
    return embeddings

In [7]:
STRATEGY_PRMOPT = """
You are a business strategy expert.
Based on the following:
1. Summary of internal documents: {local_summary}
2. Online insights and trends: {online_summary}

Generate a strategic recommendation, including:
- Risk and Opportunities
- Suggested Actions
- Market Positioning Ideas
"""

def generate_strategy(local_summary, online_summary):
    prompt = STRATEGY_PROMPT.format(local_summary=local_summary, online_summary=online_summary)
    response = openai.chat.completions.create(
        model="gpt-4",
        message=[{"role":"user", "content": prompt}]
    )
    return response.choices[0].message.content

In [4]:

generate_strategy(local_summary, online_summary)



# LLM Part I - Summarization and Part II - Strategy Modules

In [33]:
from dotenv import load_dotenv
load_dotenv()
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

# LLM Part I - Summarization

In [ ]:
'''
seperate the code into two parts, one for summarization and one for strategy modules, for
1. Modularity: for better upgrade/replace one without affecting the other
2. Reusability: The summarizer can be reuse for other tasks, such as report generation, logs, etc.
3. Focus: Each module can specialize (summarizer: compression; strategist: reasoning)
4. Pipelining: Easy to compose into an end-to-end pipeline:
   - Summarization -> Embedding -> Strategy Module
'''

In [ ]:
'''
Input: Raw text (PDF, news, platform texts, etc.
Output: Key sentences, bullet points, insights, structured results
Model Choice:
    1. OpenAI GPT-3.5/4 -> Prompt-based summarization
    2. HuggingFace Transformers (e.g., BART, T5) -> Fine-tuned for summarization tasks
    3. Custom-trained models (if domain-specific) -> Open-source models or fine-tuned on specific datasets
'''

In [ ]:
'''
Examples prompt for summarization:
Example 1:
You are an expert financial analyst. Summarize the following text into key insights, bullet points, and structured results. Focus on:
1. Key financial metrics
2. Market trends
3. Company performance
4. Risk factors

Example 2:
Given the following text, extract key sentences and bullet points that highlight the most important information. Focus on:
1. Financial performance
2. Market trends
3. Strategic insights ###
'''

In [26]:

class Summarizer:
    def __init__(self, user_authorize, db_authorize, model_name = "gpt-4"):
        self.model = model_name

    def summarize(self, text: str) -> str:
        prompt = f"Summarize the following document:\n\n{text}"
        return call_openai(prompt, self.model)

In [ ]:
summary = Summarizer().summarize(raw_text)
#forecast = trend_model.predict("AAPL")


# LLM Part II - Strategy Modules

In [ ]:
'''
Input: Summary + Trend Forecast + Market Data
Output: Natural language strategy, (todo: trading signals, risk assessment)
Model Choice:
    1. OpenAI GPT-3.5/4 -> Prompt-based summary + prediction for strategy generation
    2. Mixtral/Command R -> Open-source models for alternative strategy generation
    3. Custom fine-tuned models -> On business decision-making datasets (Optional, e.g., financial reports, market analysis)
'''

In [ ]:
'''
Examples prompt for strategy generation:
Example 1:
You are an expert financial strategist. Based on the provided summary and market data, generate a comprehensive strategy for the next quarter. Consider the following factors:
1. Market trends
2. Economic indicators
3. Company performance
4. Risk factors
Generate a detailed strategy that includes:
- Key actions to take
- Risk management strategies
- Expected outcomes

Example 2:
Given the business summary and the forecast above, suggest a short-term and long-term strategy to optimize performance. Consider cost management, regional focus, and product timeline.

'''

In [ ]:
class StrategyGenerator:
    def __init__(self, model_name="gpt-4"):
        self.model = model_name

    def generate_strategy(self, summary: str, market_data: str): # forecast: str
        prompt = (
            f"Business Summary: \n{summary}\n\n"
            # f"Forecast: \n{forecast}\n\n"
            f"Market Data: \n{market_data}\n"
            "What strategic actions should the business take next based on the above information? "
            "Provide reasoning."
            "Provide a detailed strategy that includes key actions, risk management strategies, and expected outcomes."
        )
        return call_openai(prompt, self.model)


In [ ]:
market_data = [{
    "predicting_close": 150.0
}]

In [ ]:
content = extract_text_from_pdf(pdf_files[0])
print("PDF Content Preview:\n", content[:1000])

In [ ]:
strategy = StrategyGenerator().generate(summary, market_data)